In [23]:
import pandas as pd

#pip install datasets
from datasets import Dataset
#pip install transformers
from transformers import AutoTokenizer
from transformers import BertConfig, EncoderDecoderConfig, EncoderDecoderModel
#pip install torch
from transformers import DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer


In [2]:

train_df = pd.read_csv("c_train_df_5k.csv")
test_df = pd.read_csv("c_test_df.csv")
val_df = pd.read_csv("c_dev_df.csv")

In [3]:
# Cek Indeks DataFrame
print(train_df.index)

# Cek Kolom DataFrame
print(train_df.columns)

RangeIndex(start=0, stop=57458, step=1)
Index(['Unnamed: 0', 'clean_article', 'clean_summary'], dtype='object')


In [4]:
display(train_df.head())
display(test_df.head())
display(val_df.head())

,Unnamed: 0,clean_article,clean_summary
0,0,"Liputan6. com, Indramayu : Kehebohan Alquran b...","Mohammad MM, Wahidin, dan Tanti Wydiasari meng..."
1,1,"Liputan6. com, Jakarta : Baekuni atau Babe men...","Sebelum menjebak sejumlah korbannya, Babe tern..."
2,2,"Liputan6. com, Jakarta : Bisnis bingkisan leba...",Bisnis bingkisan Lebaran di Ibu Kota kembali b...
3,3,"Liputan6. com, Makassar : Besse yang tengah ha...","Besse dan Bahir, anaknya yang berusia lima tah..."
4,4,"Liputan6. com, Medan : Ngatini alias Maulida, ...",Ngatini tewas diduga keracunan chlorofil produ...


,Unnamed: 0,clean_article,clean_summary
0,0,"Liputan6. com, Jakarta : Kasus pengeboman gere...","Sebuah bom meledak di Gereja Petra di Koja, Ja..."
1,1,"Liputan6. com, Yogyakarta : Komisi A DPRD Daer...",Mahasiswa sekolah tinggi di Yogyakarta melibat...
2,2,"Liputan6. com, Jakarta : Menyusul sejumlah aks...","Dalam sepekan, dua bom menggemparkan Ibu Kota...."
3,3,"Liputan6. com, Jakarta : Rencana pemberian abo...",Pemberian abolisi kepada Soeharto lantaran kes...
4,4,"Liputan6. com, Jakarta : Ancaman mogok besar-b...",Pemerintah akan membayar rapel gaji guru palin...


,Unnamed: 0,clean_article,clean_summary
0,0,"Liputan6. com, Surabaya : Radiogram Direktorat...",Gubernur Jatim Imam Utomo tak mau melantik Bup...
1,1,"Liputan6. com, Jakarta : Berbeda dengan aliran...",Pelukis RM Koestarto memamerkan hasil karyanya...
2,2,"Liputan6. com, Jambi : Ratusan orang dari Kesa...","Dua kelompok pengunjuk rasa di Jambi, menuntut..."
3,3,"Liputan6. com, Jakarta : Badan Penyehatan Perb...",BPPN masih mengkaji bank rekap yang dianggap p...
4,4,"Liputan6. com, Jakarta : Ketua Komisi I DPR Ya...",Kendati Dewan Papua membatalkan deklarasi keme...


In [6]:
display(train_df.info())
print('-+'*50)
display(test_df.info())
print('-+'*50)
display(val_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57458 entries, 0 to 57457
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     57458 non-null  int64 
 1   clean_article  57458 non-null  object
 2   clean_summary  57458 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


None

-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10972 entries, 0 to 10971
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     10972 non-null  int64 
 1   clean_article  10972 non-null  object
 2   clean_summary  10972 non-null  object
dtypes: int64(1), object(2)
memory usage: 257.3+ KB


None

-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10972 entries, 0 to 10971
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     10972 non-null  int64 
 1   clean_article  10972 non-null  object
 2   clean_summary  10972 non-null  object
dtypes: int64(1), object(2)
memory usage: 257.3+ KB


None

In [13]:
train_data = Dataset.from_dict(train_df)
test_data = Dataset.from_dict(test_df)
val_data = Dataset.from_dict(val_df)

In [15]:
#Menggunakan bert-tiny dengan AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

In [16]:
tokenizer.bos_token, tokenizer.eos_token, tokenizer.get_vocab()[tokenizer.bos_token], tokenizer.get_vocab()[tokenizer.eos_token]

('[CLS]', '[SEP]', 101, 102)

In [17]:
prefix = tokenizer.bos_token + " "
posfix = " " + tokenizer.eos_token

def preprocess_function(examples):
    inputs = [prefix + str(doc) + posfix for doc in examples["clean_article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(text=examples["clean_summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [18]:
tokenized_Indo_news_train = train_data.map(preprocess_function, batched=True)
tokenized_Indo_news_dev = val_data.map(preprocess_function, batched=True)
tokenized_Indo_news_test = test_data.map(preprocess_function, batched=True)

Map: 100%|██████████| 10972/10972 [00:04<00:00, 2400.68 examples/s]


In [26]:
config_encoder = BertConfig(
    vocab_size = len(tokenizer.get_vocab()), # sesuaikan dengan vocab tokenizer
    hidden_size = 36,
    num_hidden_layers = 6,
    num_attention_heads = 6,
    intermediate_size = 768,
)
config_decoder = BertConfig(
    vocab_size = len(tokenizer.get_vocab()), # sesuaikan dengan vocab tokenizer
    hidden_size = 36,
    num_hidden_layers = 6,
    num_attention_heads = 6,
    intermediate_size = 768,
)

config = EncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)
model = EncoderDecoderModel(config=config)
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id

ImportError: 
EncoderDecoderModel requires the PyTorch library but it was not found in your environment.
However, we were able to find a TensorFlow installation. TensorFlow classes begin
with "TF", but are otherwise identically named to our PyTorch classes. This
means that the TF equivalent of the class you tried to import would be "TFEncoderDecoderModel".
If you want to use TensorFlow, please use TF classes instead!

If you really do want to use PyTorch please go to
https://pytorch.org/get-started/locally/ and follow the instructions that
match your environment.


In [25]:
import torch
torch.cuda.is_available()

False

In [27]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='t5-small')

In [28]:
# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=4,
    save_total_limit=3,
    num_train_epochs=1,
    fp16=False  # Set fp16 to False
)

In [31]:
# Create and configure the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_Indo_news_train,
    eval_dataset=tokenized_Indo_news_dev,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

NameError: name 'model' is not defined